In [197]:
import numpy as np
import pandas as pd
import json
import os
import math

%config IPCompleter.greedy=True

In [198]:
data_path = '/home/ron/Documents/programovani/projekty/deepSaber/data'
os.chdir(data_path)
output_shape = (3, 4, 2, 9)  # width × height × color × rotation (last == any)

hard = 'Kneel Before Me/Normal.json'
extreme_plus = '[A]ddiction/ExpertPlus.json'
with open(extreme_plus) as json_data:
    data = json.load(json_data)

data

{'_version': '1.5.0',
 '_beatsPerMinute': 164,
 '_beatsPerBar': 16,
 '_noteJumpSpeed': 12,
 '_shuffle': 0,
 '_shufflePeriod': 0.5,
 '_time': 553,
 '_events': [{'_time': 0.8826666474342346, '_type': 4, '_value': 2},
  {'_time': 3.38266658782959, '_type': 13, '_value': 4},
  {'_time': 3.38266658782959, '_type': 2, '_value': 6},
  {'_time': 3.38266658782959, '_type': 12, '_value': 4},
  {'_time': 6.38266658782959, '_type': 3, '_value': 2},
  {'_time': 6.38266658782959, '_type': 0, '_value': 7},
  {'_time': 6.38266658782959, '_type': 1, '_value': 2},
  {'_time': 6.38266658782959, '_type': 8, '_value': 0},
  {'_time': 11.38266658782959, '_type': 3, '_value': 0},
  {'_time': 11.38266658782959, '_type': 2, '_value': 0},
  {'_time': 12.38266658782959, '_type': 2, '_value': 7},
  {'_time': 12.38266658782959, '_type': 13, '_value': 0},
  {'_time': 12.38266658782959, '_type': 12, '_value': 0},
  {'_time': 12.38266658782959, '_type': 3, '_value': 7},
  {'_time': 13.38266658782959, '_type': 0, '_va

In [199]:
df = pd.DataFrame(data['_notes'])
df

,_cutDirection,_lineIndex,_lineLayer,_time,_type
0,2,0,1,6.382667,0
1,3,3,1,6.382667,1
2,1,1,0,7.382667,0
3,0,1,0,7.882667,0
4,1,2,0,8.382667,1
5,1,1,0,8.882667,0
6,0,0,0,9.382667,0
7,0,3,0,9.382667,1
8,1,3,0,10.382667,1
9,0,3,0,10.882667,1


In [200]:
(df.apply(lambda x: int(100 * (x['_time'] - math.floor(x['_time']))), axis=1) / 100).value_counts()


0.38    639
0.88    245
0.63      1
0.13      1
dtype: int64

- Most of the blocks appear precisely on the beats.
- Some are on half beats.
- From Expert and harder there are even triolas and wierd stuff

In [201]:
arr = np.zeros(output_shape)
arr[2][3][1][8]

0.0

In [202]:
df['_time'] = round(df['_time'], 2)

def create_ndarr(data):
    arr = np.zeros(output_shape)
    
    for index, row in data.iterrows():
        arr[row['_lineLayer'], row['_lineIndex'], row['_type'], row['_cutDirection']] = 1
    return arr

new_df = pd.DataFrame(df.groupby('_time').apply(create_ndarr), columns=['output'])

In [203]:
new_df['prev'] = 0
last_time = 0

for i, row in new_df.iterrows():
    new_df.loc[i, 'prev'] = i - last_time
    last_time = i

new_df['next'] = new_df['prev'].shift(periods=-1).fillna(0)
new_df

,output,prev,next
_time,,,
6.38,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",6.38,1.0
7.38,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",1.00,0.5
7.88,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",0.50,0.5
8.38,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",0.50,0.5
8.88,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",0.50,0.5
9.38,"[[[[1. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",0.50,1.0
10.38,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",1.00,0.5
10.88,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",0.50,0.5
11.38,"[[[[0. 0. 0. 0. 0. 0. 0. 0. 0.], [0. 0. 0. 0. ...",0.50,1.0


In [234]:
sr = pd.Series([1] * 21 + [2] * 21 )
df = pd.DataFrame({'ASD': sr * 2, 'B': sr})
df['out'] = df.loc[~(df['ASD'] == 4)].apply(lambda x: 3, axis=1)
df

,ASD,B,out
0,2,1,3.0
1,2,1,3.0
2,2,1,3.0
3,2,1,3.0
4,2,1,3.0
5,2,1,3.0
6,2,1,3.0
7,2,1,3.0
8,2,1,3.0
9,2,1,3.0
